# <font color='red'>Project IRIS: with Keras+sklearn</font>

In this tutorial you will discover **how to use Keras+sklearn to develop and evaluate a NN model for a multiclass classification problem**. 

Goals:
* How to load data from CSV and make it available to Keras
* How to prepare multiclass classification data for modeling with NNs
* How to evaluate Keras NN models with scikit-learn

# Description of the input data

The iris flowers dataset is a standard ML dataset, widely used worldwide as benchmark.

### Dataset availability:

Almost ubiquitous.. e.g.
   * [UCI Machine Learning repository](http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data)

More info:
   * [UCI Machine Learning Repository page](https://archive.ics.uci.edu/ml/datasets/Iris).


### Dataset description:

* This is a good example to practice on a multiclass classification problem.
* Each instance describes the properties of an observed iris flower measurements
* All of the 4 input variables are numeric and have the same scale (cm)
   * Sepal length in centimeters 
   * Sepal width in centimeters 
   * Petal length in centimeters 
   * Petal width in centimeters
* The output variable is a specific iris species (3 possibilities)
   * the "class", e.g. "Iris-setosa", "Iris-versicolor" or "Iris-verginica"

### How the dataset looks like:


    5.1,3.5,1.4,0.2,Iris-setosa
    4.9,3.0,1.4,0.2,Iris-setosa
    4.7,3.2,1.3,0.2,Iris-setosa
    4.6,3.1,1.5,0.2,Iris-setosa
    5.0,3.6,1.4,0.2,Iris-setosa
    (...)


### Additional input from best practitioners:

The iris flower dataset is a well studied problem and as such we can expect to achieve a model accuracy in the range of 95% to 97%. USe this as target to aim for when developing your model(s).

# Start-up

## Import Classes and Functions

Start by importing all classes and functions you will need:
* all the functionality we require from **Keras**
* data loading functionalities from **Pandas**
* data preparation and model evaluation from **scikit-learn**
* more as needed (e.g. **numpy**)

In [3]:
import numpy
# pandas
from pandas import read_csv
# keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
# sklearn
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


## Initialize random nb generator

Important to ensure that the results we achieve from this model are repeatable, i.e. it ensures that the stochastic process of training a NN model can be reproduced.

In [5]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

## Load The Dataset

Download the dataset from the link above and place it in your current working directory, with filename $iris.data.csv$. 

Then, you can quickly inspect it from this same ipynb.

In [6]:
!ls -trl iris.data.csv

-rw-r--r--@ 1 bonacor  staff  4551 May 12 11:22 iris.data.csv


In [7]:
!head -5 iris.data.csv

5.1,3.5,1.4,0.2,Iris-setosa
4.9,3.0,1.4,0.2,Iris-setosa
4.7,3.2,1.3,0.2,Iris-setosa
4.6,3.1,1.5,0.2,Iris-setosa
5.0,3.6,1.4,0.2,Iris-setosa


Now, load the dataset. You can do it in various ways. The output variable contains strings, so it is suggested (easiest) to load the data using **pandas** into a DataFrame. While you do so, split the attributes (i.e. columns) into input variables (the matrix of **features X**) and output variables (the vector **label Y**).

In [8]:
# load dataset
dataframe = read_csv("iris.data.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)   # columns from 1st to 4th into X
Y = dataset[:,4]                   # column 5th into Y

Verify what you did.

In [14]:
len(X)

150

In [15]:
len(Y)

150

In [16]:
X

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [10]:
Y

array(['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor',
       'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor',
       'Iris-versicolor', 'Iris-versicolor', 'Iris-versic

# Data preparation/preprocessing

## One-hot encoding the output variable

*(NOTE: different datasets may require different data manipulation/preprocessing. This applies to this specific case)*

When modeling multiclass classification problems using NNs it is practice to reshape the output attribute from a vector that contains values for each class value to a matrix with a boolean for each class value and whether or not a given instance has that class value or not. 

In my case, the output variable contains 3 different class (string) values, i.e. I can have one of each of these 3 observations:

    Iris-setosa
    Iris-versicolor
    Iris-virginica
    
I translate this into a one hot encoded binary matrix for each data instance among these 3:

    Iris-setosa, Iris-versicolor, Iris-virginica


that would hence look as follows:

    1, 0, 0
    0, 1, 0
    0, 0, 1
    
We can do this easily by:
   1. encoding the strings consistently to integers using the scikit-learn class *LabelEncoder*
   2. convert the vector of integers to a one-hot encoding using the Keras function *to_categorical()*

In [18]:
# part 1: encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

Check:

In [22]:
encoder

LabelEncoder()

In [29]:
encoded_Y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [30]:
# part 2: do one-hot encoding
transformed_Y = np_utils.to_categorical(encoded_Y)

In [31]:
transformed_Y

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0

# Define a NN model

## Create a baseline model

You can create a baseline NN - a simple **Fully Connected NN (FCNN)** - for the IRIS multiclass classification problem with just one function:
   * input
       * as per our input dataset, this NN has 4 inputs (X)
   * hidden layer(s)
       * the hidden layer here has 8 nodes, and uses a rectifier (**relu**) activation function, which is a good practice
   * output
       * because we used a one-hot encoding for the dataset, the output layer must create 3 output values, one for each class. We use a **softmax** activation function in the output layer, to ensure the output values are in the range of 0 and 1 and may be used as predicted probabilities: the output value with the largest value will be taken as the class predicted by the model. Finally, the network uses the efficient **adam** GD optimization algorithm with a **logarithmic loss function**, which is called **categorical crossentropy** in Keras.   
   
Hence, the network topology of this simple 1-layer FCNN can be summarized as:

    4 inputs -> 1 hidden layer with 8 nodes -> 3 outputs

and it simplementation in Keras is as simple as follows:

In [32]:
# define a baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=4, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [35]:
baseline_model()

## Use KerasClassifier


The idea here is to use the Keras library which provides wrapper classes to allow you to use NN models developed with Keras (see cell above) in scikit-learn. Why so? Because Keras is simple, and scikit-learn is powerful and versatile!

There is a *KerasClassifier* class in Keras that can be used as an *Estimator* in scikit-learn, the base type of model in the library. We need to actually create our KerasClassifier first, to be used in scikit-learn. KerasClassifier takes the name of a function (the one we wrote above) as an argument, plus arguments that will be passed on to the *fit()* function internally used to train the NN. Here, we pass:

* a number of epochs as 200
* a batch size as 5 

to use when training the model. Debugging is also turned off when training by setting verbose to 0.
    
This function returns the constructed NN model, ready for training.

In [37]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)

In [39]:
estimator

## Evaluate The Model with $k$-Fold Cross-Validation

It is time to evaluate our NN model on our training data, a.k.a. the "training" phase.

The scikit-learn library has excellent capability to evaluate models using a suite of techniques. The gold standard for evaluating ML models is **k-fold cross-validation (k-fold CV)**. We do as follows:

1. we define the model evaluation procedure.
      * here, we shuffle the data before partitioning it, and we set the number of folds to 10 (a good default)
     
     
2. we evaluate our model (*estimator*) on our dataset (*X* and *transformed_Y*) using a 10-fold CV procedure (kfold)

Evaluating the model only takes approximately 10 seconds and returns an object that describes the evaluation of the k=10 constructed models for each of the splits of the dataset. 

The results are summarized as both the mean and standard deviation of the model accuracy on the dataset.

In [40]:
# part 1
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [41]:
# part 2
results = cross_val_score(estimator, X, transformed_Y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 97.33% (4.42%)


This is a reasonable estimation of the performance of the model on unseen data. It is also within the realm of known top results for this problem.

# Summary 

What we learned:

* how to develop and evaluate a NN using the Keras library for ML/DL.

Specifically:

* How to load data and make it available to Keras
* How to prepare multiclass classification data for modeling using one hot encoding
* How to use Keras NN models with scikit-learn
* How to define a NN using Keras for multiclass classification
* How to evaluate a Keras NN model using scikit-learn with k-fold cross-validation